In [1]:
%matplotlib inline

In [20]:
import pandas
import src
import gensim
import os
import os.path
import csv

In [66]:
rankfiles = dict()
for dirpath, dirname, filenames in os.walk('runs'):
    for filename in filenames:
        if filename.endswith("-ranks.csv.gz"):
            _, num, _, name, version = dirpath.split('/')
            if name not in rankfiles:
                rankfiles[name] = {'changeset': {'feature_location': list(),
                                                 'triage': list()},
                                   'release': {'feature_location': list(),
                                               'triage': list()},
                                  }
                
            parts = filename.split('-')
            rankfiles[name][parts[0]][parts[1]].append(os.path.join(dirpath, filename))

In [67]:
    path = rankfiles['bookkeeper']['release']['triage'][0]
    ranks = dict()
    with gensim.utils.smart_open(path) as f:
        reader = csv.reader(f)
        next(reader)  # skip header
        for g_id, idx, dist, d_name in reader:
            if g_id not in ranks:
                ranks[g_id] = list()

            ranks[g_id].append((int(idx), float(dist), gensim.utils.to_unicode(d_name)))

In [68]:
gs = src.triage.create_goldsets(bk)

2015-12-06 18:18:15 bocmaxima common[2471] INFO Loading goldsets for project: Project(name='bookkeeper', printable_name='bookkeeper v4.3.0', version='v4.3.0', ref='refs/tags/release-4.3.0', data_path='data/bookkeeper/', full_path='data/bookkeeper/v4.3.0/', src_path='data/bookkeeper/v4.3.0/src/', level='file')
2015-12-06 18:18:15 bocmaxima common[2471] INFO Loading issue2git.csv
2015-12-06 18:18:15 bocmaxima common[2471] INFO Loaded issue2git with 572 entries
2015-12-06 18:18:15 bocmaxima common[2471] INFO Returning issue2git with len 572 and git2issue with len 730
2015-12-06 18:18:16 bocmaxima common[2471] INFO Returning 164 goldsets


In [69]:
src.utils.calculate_mrr([x for x, y,z in src.common.get_frms(ranks, gs)])

2015-12-06 18:18:18 bocmaxima common[2471] INFO Getting FRMS for 164 ranks
2015-12-06 18:18:18 bocmaxima common[2471] INFO Returning 164 FRMS


0.67164634146341451

In [113]:
df = pandas.DataFrame()
for name, source in rankfiles.items():
    for source, exps in source.items():
        for exp, filenames in exps.items():
            mrr = list()
            for each in filenames:
                data = pandas.read_csv(each)
                mrr.append(src.utils.calculate_mrr(list(data.groupby("id")["rank"].min())))
        
            mrr = pandas.Series(mrr)
            if mrr.std() != 0.0:
                df = df.append(
                  {'name': name, 
                   'source': source,
                   'experiment': exp,
                   'std': mrr.std(),
                   'max-min': mrr.max() - mrr.min(),
                   'max': mrr.max(),
                   'min': mrr.min(),
                   'count': mrr.count(),
                   'raw': mrr
                  }, ignore_index=True)

In [114]:
df.where((df.experiment == 'feature_location') & (df.source == 'release')).sort_values(by=['max-min', 'std']).dropna()

,count,experiment,max,max-min,min,name,raw,source,std
0,25,feature_location,0.231256,0.075217,0.156039,openjpa,0 0.188347 1 0.193732 2 0.189910 3...,release,0.017172
3,36,feature_location,0.433985,0.083861,0.350124,zookeeper,0 0.370326 1 0.396219 2 0.384131 3...,release,0.016100
15,74,feature_location,0.559250,0.086100,0.473149,bookkeeper,0 0.559009 1 0.495146 2 0.517076 3...,release,0.021126
6,23,feature_location,0.385316,0.105743,0.279573,pig,0 0.335448 1 0.346491 2 0.279573 3...,release,0.024304
9,26,feature_location,0.318799,0.116797,0.202002,mahout,0 0.309602 1 0.284767 2 0.318799 3...,release,0.031074
12,39,feature_location,0.448330,0.214335,0.233995,tika,0 0.328147 1 0.301087 2 0.348143 3...,release,0.048770
